In [1]:
data_var = ''

In [2]:
# Parameters
data_var = "2024-05-17"


In [3]:
import pandas as pd; pd.set_option('display.max_columns', None)
import os
from pycaret.classification import *
from scripts.PreparaDatasetV2 import *
from scripts.TeoriaRetornos import *

### Funções

In [4]:
def atualiza_dia(_jogos):
    _jogos = prepara_df(_jogos)

    _jogos = _jogos.reset_index(drop=True)

    return _jogos

### Pegando os jogos do dia

In [5]:
df_today = pd.read_csv(f'jogos_do_dia/{data_var}.csv')

### Atualizando os datasets que tem jogo hoje

In [6]:
temporada = pd.read_csv('data/Entire_Season.csv')
temporada = temporada[temporada['Date'] < data_var]
temporada = pd.concat([temporada, df_today])
ligas = df_today['League'].unique().tolist()
temporada = temporada[temporada['League'].isin(ligas)]
temporada.reset_index(drop=True, inplace=True)

### Cria variáveis na temporada e nos jogos do dia

In [7]:
games = atualiza_dia(temporada)
temporada = games[games['Date'] < data_var]
os.makedirs(f'apostas_do_dia/{data_var}', exist_ok=True)

In [8]:
jogos = games[games['Date'] == data_var]

jogos = jogos.drop(columns=['Home_Pts', 'Away_Pts',
       'Back_Home', 'PL_Home', 'Back_Away',
       'PL_Away', 'Back_Over', 'PL_Over', 'PL_Under', 'Back_HA_H', 'PL_HA_H',
       'PL_HA_A', ])

jogos

,Date,League,Time,Home,Away,Odds_H,Odds_A,Over_Line,Odds_Over,Odds_Under,HA_Line,HA_Odds_H,HA_Odds_A,Fixture ID,P(H),P(A),P(O),P(U),P_Diff,Avg_CG_H,DP_CG_H,CV_CG_H,Avg_Pontos_H,DP_Pontos_H,CV_Pontos_H,CG_H_Last_Game,Media_Odds_Geral_H,DP_Odds_Geral_H,CV_Odds_Geral_H,Saldo_Gols_H,Avg_CG_A,DP_CG_A,CV_CG_A,Avg_Pontos_A,DP_Pontos_A,CV_Pontos_A,CG_A_Last_Game,Media_Odds_Geral_A,DP_Odds_Geral_A,CV_Odds_Geral_A,Saldo_Gols_A,Goals_Last_H,Goals_Last_A,Last_Odd_H,Last_Odd_A,MediaCustoGolHome,MediaCustoGolAway,CV_ML,CV_Over,CV_HA,Retornos_BH_Acu,Avg_Retornos_BH,Custo_Retorno_BH,Avg_Porc_BH_Bookie,Avg_Porc_BH_Real,Dist_Porc_BH,Retornos_BA_Acu,Avg_Retornos_BA,Custo_Retorno_BA,Avg_Porc_BA_Bookie,Avg_Porc_BA_Real,Dist_Porc_BA
12437,2024-05-17,Países Baixos Dbl,14:30,Groningen,Den Bosch,2.31,1.57,151.5,1.88,1.79,0.0,0.0,0.0,0EoYQlI8,0.432900,0.636943,0.531915,0.558659,0.069843,0.000,0.000000,0.000000,0.0,0.000000,0.000000,0.00,0.000,0.000000,0.000000,0.0,0.000,0.000000,0.000000,0.0,0.000000,0.000000,0.00,0.000,0.000000,0.000000,0.0,79,90,5.93,1.11,0.000,0.000,0.269721,0.034681,NaN,0.00,0.000,inf,0.000000,0.0,0.000000,0.00,0.000,inf,0.000000,0.0,0.000000
12438,2024-05-17,Romênia Divisão A,12:00,Laguna Bucuresti,CSM Targu Jiu,3.12,1.33,161.5,1.86,1.84,0.0,0.0,0.0,QFe8q4iL,0.320513,0.751880,0.537634,0.543478,0.072393,546.650,211.218803,0.386388,0.6,1.341641,2.236068,672.39,6.916,1.966871,0.284394,-59.0,121.946,33.881309,0.277839,0.6,1.341641,2.236068,113.42,1.404,0.404388,0.288026,-26.0,93,106,7.23,1.07,444.580,157.366,0.568863,0.007644,NaN,2.28,0.456,4.649123,0.000000,0.0,0.000000,-5.00,-1.000,-0.330000,0.389606,0.2,-0.189606
12439,2024-05-17,Portugal Lpb,17:00,Oliveirense,CD Póvoa,1.21,4.12,157.5,1.85,1.85,0.0,0.0,0.0,jZvjZqeo,0.826446,0.242718,0.540541,0.540541,0.069165,161.968,111.129205,0.686118,2.4,1.341641,0.559017,98.28,1.922,1.276820,0.664319,38.0,319.142,344.761846,1.080277,1.2,1.643168,1.369306,234.65,4.164,4.488154,1.077847,-32.0,78,65,1.26,3.61,130.728,435.858,0.772113,0.000000,NaN,1.61,0.322,0.652174,0.688233,0.7,0.011767,7.75,1.550,2.012903,0.000000,0.0,0.000000
12440,2024-05-17,Países Baixos Dbl,14:30,Zwolle,Leiden,1.54,2.37,145.5,1.81,1.85,0.0,0.0,0.0,b7Sc01vS,0.649351,0.421941,0.552486,0.540541,0.071292,0.000,0.000000,0.000000,0.0,0.000000,0.000000,0.00,0.000,0.000000,0.000000,0.0,0.000,0.000000,0.000000,0.0,0.000000,0.000000,0.00,0.000,0.000000,0.000000,0.0,60,68,2.29,1.58,0.000,0.000,0.300204,0.015456,NaN,0.00,0.000,inf,0.000000,0.0,0.000000,0.00,0.000,inf,0.000000,0.0,0.000000
12441,2024-05-17,Romênia Divisão A,13:30,Arges Pitesti,Voluntari,6.20,1.10,158.5,1.84,1.84,0.0,0.0,0.0,2u97EmkT,0.161290,0.909091,0.543478,0.543478,0.070381,435.192,249.243927,0.572722,0.0,0.000000,NaN,201.50,6.156,3.289062,0.534286,-114.0,157.680,41.773101,0.264923,2.4,1.341641,0.559017,125.76,1.886,0.647441,0.343288,52.0,62,96,3.25,1.31,220.574,203.404,0.988012,0.000000,NaN,-2.13,-0.426,-12.206573,0.000000,0.0,0.000000,-3.30,-0.660,-0.151515,0.468791,0.4,-0.068791
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12503,2024-05-17,Chile Lnb,21:30,CD Puerto Varas,Las Animas,2.68,1.43,165.5,1.86,1.80,0.0,0.0,0.0,r5175Yig,0.373134,0.699301,0.537634,0.555556,0.072435,178.396,75.886810,0.425384,0.6,1.341641,2.236068,117.26,2.538,1.218799,0.480220,-46.0,152.302,56.665679,0.372061,1.2,1.643168,1.369306,225.12,1.968,0.786937,0.399866,-5.0,82,84,1.43,2.68,177.230,317.678,0.430114,0.023184,NaN,-2.59,-0.518,-3.243243,0.521870,0.5,-0.021870,-3.75,-0.750,-0.573333,0.453444,0.3,-0.153444
12504,2024-05-17,Bulgária Nbl,13:00,Rilski Sportist,Spartak Pleven,1.29,3.37,163.5,1.86,1.80,0.0,0.0,0.0,lbOivGxR,0.775194,0.296736,0.537634,0.555556,0.071930,117.258,26.585905,0.226730,2.4,1.341641,0.559017,155.70,1.270,0.274500,0.216141,95.0,189.696,109.992820,0.579837,1.2,1.643168

## Apostas do dia

### Back Home V3 (QCut)

In [9]:
# Duplicando o dataset
df_bh_v3 = jogos.copy()

df_bh_v3 = df_bh_v3[(df_bh_v3['P(H)'] > 0.35) & (df_bh_v3['P(H)'] < 0.65)]
df_bh_v3 = df_bh_v3[(df_bh_v3['CG_H_Last_Game'] > 0) & (df_bh_v3['CG_H_Last_Game'] <= 170)]
df_bh_v3 = df_bh_v3[(df_bh_v3['CG_A_Last_Game'] > 160)]

df_bh_v3.replace([np.inf, -np.inf], 0, inplace=True)

if not df_bh_v3.empty:
    df_bh_v3.to_csv(f'apostas_do_dia/{data_var}/back_home_v3_{data_var}.csv', index=False)

    print('------------ MÉTODO BACK HOME V3 (QCUT) ------------')
    print()

    df_bh_v3['Bet'] = 'Back Home'
    ligs = df_bh_v3['League'].unique().tolist()

    display(df_bh_v3[['Time', 'League', 'Home', 'Away', 'Odds_H', 'Bet']])
    print()
else: print('Sem jogos')

------------ MÉTODO BACK HOME V3 (QCUT) ------------



,Time,League,Home,Away,Odds_H,Bet
12442,12:50,Lituânia Lkl,Juventus,Lietkabelis,1.75,Back Home
12495,13:00,Estônia Korvpalli Meistriliiga,Pärnu,Rapla,1.58,Back Home
12503,21:30,Chile Lnb,CD Puerto Varas,Las Animas,2.68,Back Home
